<a href="https://colab.research.google.com/github/llpk79/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/Paul_K_Sprint_Challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [0]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data'

In [0]:
columns = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16']
data = pd.read_csv(url, header=None, names=columns, na_values='?')
data = data.fillna(method='bfill')
data.isna().sum()

In [28]:
data.describe(include='object')

,A1,A4,A5,A6,A7,A9,A10,A12,A13,A16
count,690,690,690,690,690,690,690,690,690,690
unique,2,3,3,14,9,2,2,2,3,2
top,b,u,g,c,v,t,f,f,g,-
freq,475,523,523,140,407,361,395,374,625,383


## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" (aka [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)) between them and the class label, and apply the Chi-squared test to them. [pandas.crosstab](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) can create contingency tables, and [scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) can calculate the Chi-squared statistic for them.

There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end - use simple visualizations, such as boxplots or a scatter matrix (both built in to pandas), to get a feel for the overall distribution of the variables.

This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [0]:
pos = data[data['A16'] == '+']
neg = data[data['A16'] == '-']

In [0]:
pos.describe()

In [0]:
neg.describe()

In [0]:
cont_columns = ['A2',	'A3',	'A8',	'A11',	'A14',	'A15']

In [27]:
for column in cont_columns:
  stat, pval = st.ttest_ind(pos[column], neg[column])
  print(column, stat, pval)

A2 4.426470508079215 1.1136636133412522e-05
A3 5.52998337614816 4.551680702308068e-08
A8 8.935819983773698 3.6710537401601785e-18
A11 11.667004222431277 7.957718568079967e-29
A14 -2.778766584532883 0.005605436656742499
A15 4.680216020964486 3.4520256956287944e-06


Most are "extremely" different as evidenced by the very low P values, with the exception of maybe A14, as I don't know what qualifies as extreme, but it is less different.

In [0]:
cat_columns = ['A1',	'A4',	'A5',	'A6',	'A7',	'A9',	'A10',	'A12',	'A13']

In [35]:
for column in cat_columns:
  tab = pd.crosstab(index=pos['A16'], columns=pos[column])
  chi_squared, p_value, dof, expected = st.chi2_contingency(tab)
  print(column, p_value)

A1 0.6384724108987662
A4 7.998275136625324e-07
A5 7.998275136625324e-07
A6 1.6060658740367989e-15
A7 2.5758929132392513e-07
A9 3.1185900878457007e-79
A10 5.675727374527571e-33
A12 0.45094587758631943
A13 0.010094291370456362


We reject the null hypothesis of independence with A1 and A12. They are on the same order of magnetude of "extremeness" in their dependence with A16.

In [39]:
for column in cat_columns:
  tab = pd.crosstab(index=neg['A16'], columns=neg[column])
  chi_squared, p_value, dof, expected = st.chi2_contingency(tab)
  print(column, p_value)

A1 1.0
A4 1.0
A5 1.0
A6 1.0
A7 1.0
A9 1.0
A10 1.0
A12 1.0
A13 1.0


There are apparently no independent variables among the rejected applications, but this just makes me think I'm missing something major.

In [0]:
for column in columns[:-1]:
  tab = pd.crosstab(index=data['A16'], columns=data[column])
  chi_squared, p_value, dof, expected = st.chi2_contingency(tab)
  print(column, p_value)

## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

- I'll go with A11 and A14. The p_value for A11 is very small, so we conclude that the neg and pos groups are significantly different on column A11. A14 has a much higher p_value so we cannot reject the null hypothesis that they are not different.

- I'm pretty sure I've screwed the chi2 test up somehow, but, within the pos group, A1 and A12 are not independent from A16. Within the neg groups all appear to be dependent on A16. 

- The most challenging part of this is that I feel like this was rushed through in lectures with broken and simplistic examples and I never got very thourough answers to questions in office hours. I feel like I should probably repeat this week in flex.

  The lack of meaningful labels is also challenging.